In [14]:
import os
import time
import pandas as pd
import urllib.request
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException


def scroll_shim(passed_in_driver, object):
    x = object.location['x']
    y = object.location['y']
    scroll_by_coord = 'window.scrollTo(%s,%s);' % ( x, y)
    scroll_nav_out_of_way = 'window.scrollBy(0, -73);'
    passed_in_driver.execute_script(scroll_by_coord)
    passed_in_driver.execute_script(scroll_nav_out_of_way)
    
def feature_function(features):
    features_list=features.split("\n")
    long=len(features_list)
    count_till= 40  
    while long < count_till:
        features_list.append(float('nan'))
        long+=1
    return features_list



In [15]:
ean_list= ['2595348013662', '8710755385742']

In [18]:
def eci(ean_list):
    
    eci_info = pd.DataFrame()
    
    for ean in ean_list:

        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'

        browser = webdriver.Firefox()
        browser.get('https://www.elcorteingles.es/')

        search = browser.find_element_by_xpath('//input[@id="search-box"]')
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(4)

        title = browser.find_elements_by_xpath('//h2[@class="title"]')[0].text
        price = browser.find_elements_by_xpath('//div[@class="product-price "]')[0].text
        price_list=price.split("€")

        '''
        customer_reviews = browser.find_elements_by_xpath('//button[@id="star-button"]')
        for my_aria_label in customer_reviews:
            customer_reviews =my_aria_label.get_attribute("aria-label")

        customer_reviews = browser.find_elements_by_xpath('//button[@id="star-button"]').get_attribute('aria-label')
        '''

        main_image = browser.find_element_by_xpath('//img[@id="product-image-placer"]').get_attribute('src')

        try:

            action = ActionChains(browser)
            click_view_more  = browser.find_element_by_xpath("(//span[@id='text-view-more'])")
            if 'firefox' in browser.capabilities['browserName']:
                scroll_shim(browser, click_view_more)
            action.move_to_element((click_view_more)).click().perform()

            features = browser.find_elements_by_xpath('//div[@id="features"]')[0].text
            features_list=feature_function(features)
            more_features = browser.find_elements_by_xpath('//div[@id="inpage_container"]')[0].text
            more_features_list=more_features.split("\n")

            eci_data = pd.DataFrame({'ean.value': [ean],
                                         'item_name.value': [title], 
                                         'list_price.value_with_tax': [price_list[0]],
                                         'short_product_description.value': features_list[0],
                                         f'{features_list[1]}': [features_list[2]],
                                         f'{features_list[3]}': [features_list[4]],
                                         f'{features_list[5]}': [features_list[6]],
                                         f'{features_list[7]}': [features_list[8]],
                                         f'{features_list[9]}': [features_list[10]],
                                         f'{features_list[11]}': [features_list[12]],
                                         f'{features_list[13]}': [features_list[14]],
                                         f'{features_list[15]}': [features_list[16]],
                                         f'{features_list[17]}': [features_list[18]],
                                         f'{features_list[19]}': [features_list[20]],
                                         f'{features_list[21]}': [features_list[22]],
                                         f'{features_list[23]}': [features_list[24]],
                                         f'{features_list[25]}': [features_list[26]],
                                         f'{features_list[27]}': [features_list[28]],
                                         f'{features_list[29]}': [features_list[30]],
                                         f'{features_list[31]}': [features_list[33]],
                                         'more_features': [more_features_list],
                                         'main_image': [main_image]
                                        })
            
            browser.close()
            eci_data = eci_data.drop(columns=['nan'])
            eci_data = eci_data.set_index('ean.value')
            eci_info = pd.concat([eci_info, eci_data], axis=1, sort=False)

            

        except NoSuchElementException: 

            try:

                features = browser.find_elements_by_xpath('//div[@id="features"]')[0].text
                features_list=feature_function(features)
                more_features = browser.find_elements_by_xpath('//div[@id="inpage_container"]')[0].text
                more_features_list=more_features.split("\n")

                eci_data = pd.DataFrame({'ean.value': [ean],
                                             'item_name.value': [title], 
                                             'list_price.value_with_tax': [price_list[0]],
                                             'short_product_description.value': features_list[0],
                                             f'{features_list[1]}': [features_list[2]],
                                             f'{features_list[3]}': [features_list[4]],
                                             f'{features_list[5]}': [features_list[6]],
                                             f'{features_list[7]}': [features_list[8]],
                                             f'{features_list[9]}': [features_list[10]],
                                             f'{features_list[11]}': [features_list[12]],
                                             f'{features_list[13]}': [features_list[14]],
                                             f'{features_list[15]}': [features_list[16]],
                                             f'{features_list[17]}': [features_list[18]],
                                             f'{features_list[19]}': [features_list[20]],
                                             f'{features_list[21]}': [features_list[22]],
                                             f'{features_list[23]}': [features_list[24]],
                                             f'{features_list[25]}': [features_list[26]],
                                             f'{features_list[27]}': [features_list[28]],
                                             f'{features_list[29]}': [features_list[30]],
                                             f'{features_list[31]}': [features_list[33]],
                                             'more_features': [more_features_list],
                                             'main_image': [main_image]
                                            })
        
                browser.close()
                eci_data = eci_data.drop(columns=['nan'])
                eci_data = eci_data.set_index('ean.value')
                eci_info = pd.concat([eci_info, eci_data], axis=1, sort=False)
    

            except IndexError:

                features = browser.find_elements_by_xpath('//div[@id="features"]')[0].text
                features_list=feature_function(features)

                eci_data = pd.DataFrame({'ean.value': [ean],
                                             'item_name.value': [title], 
                                             'list_price.value_with_tax': [price_list[0]],
                                             'short_product_description.value': features_list[0],
                                             f'{features_list[1]}': [features_list[2]],
                                             f'{features_list[3]}': [features_list[4]],
                                             f'{features_list[5]}': [features_list[6]],
                                             f'{features_list[7]}': [features_list[8]],
                                             f'{features_list[9]}': [features_list[10]],
                                             f'{features_list[11]}': [features_list[12]],
                                             f'{features_list[13]}': [features_list[14]],
                                             f'{features_list[15]}': [features_list[16]],
                                             f'{features_list[17]}': [features_list[18]],
                                             f'{features_list[19]}': [features_list[20]],
                                             f'{features_list[21]}': [features_list[22]],
                                             f'{features_list[23]}': [features_list[24]],
                                             f'{features_list[25]}': [features_list[26]],
                                             f'{features_list[27]}': [features_list[28]],
                                             f'{features_list[29]}': [features_list[30]],
                                             f'{features_list[31]}': [features_list[33]],
                                             'main_image': [main_image]
                                            })
                
                browser.close()
                eci_data = eci_data.drop(columns=['nan'])
                eci_data = eci_data.set_index('ean.value')
                eci_info = pd.concat([eci_info, eci_data], axis=1, sort=False)
    
    return eci_info, eci_data
                



In [19]:
df,df1 = eci(ean_list)